In [52]:
import sys
sys.path.append('polytrader/')

In [53]:
from agents.polymarket.polymarket import Polymarket

In [54]:
p = Polymarket()

In [55]:
print(p.get_address_for_private_key())

0x3D68130CA229e08b9245be1CC8AD10Bb5889dEEE


In [56]:
markets = p.client.get_markets()

In [57]:
for market in markets['data']:
    if market['accepting_orders']:
        print('Market: \n')
        print(market['question'])
        print('\n')

Market: 

Will the Fed cut rates in 2023?


Market: 

Will the FDV of OpenSea's token 1 week after launch be above Blur's FDV?


Market: 

Will there be a Senate vacancy in Pennsylvania by EOY?


Market: 

Will the FDV of OpenSea's token be above $5b 1 week after launch?


Market: 

Will Trump win the 2024 Iowa Caucus?


Market: 

Will the FDV of OpenSea's token be above $15b 1 week after launch?


Market: 

Will GPT-4 have 500b+ parameters?


Market: 

Will Julie Su be the next U.S. Secretary of Labor?


Market: 

Will Tim Ryan be the next Secretary of Labor?


Market: 

Will Sean Patrick Maloney be the next Secretary of Labor?


Market: 

Will the FDV of OpenSea's token be above $10b 1 week after launch?


Market: 

Will Andy Levin be the next Secretary of Labor?




In [58]:
big_L = p.client.get_trades()[0]['market']

In [59]:
p.client.get_market(big_L)

{'enable_order_book': False,
 'active': True,
 'closed': True,
 'archived': False,
 'accepting_orders': False,
 'accepting_order_timestamp': '2024-11-24T18:58:13Z',
 'minimum_order_size': 5,
 'minimum_tick_size': 0.001,
 'condition_id': '0x15d7d9526d1c5ac0f108e985affcbcc3ef7cc95b8fa34cb3f8c07bddd7063147',
 'question_id': '0xc6a2367122685775e80838bc1288aff85b94aaad93955cb0a37b96266ea95942',
 'question': 'Cavaliers vs. Hawks',
 'description': 'In the upcoming NBA game, scheduled for November 29 at 2:30PM ET:\nIf the Cleveland Cavaliers win, the market will resolve to “Cavaliers”.\nIf the Atlanta Hawks win, the market will resolve to “Hawks”.\nIf the game is not completed by December 6, 2024 (11:59 PM ET) or it ends in a tie, the market will resolve 50-50.',
 'market_slug': 'nba-cle-atl-2024-11-29',
 'end_date_iso': '2024-12-06T00:00:00Z',
 'game_start_time': '2024-11-29T19:30:00Z',
 'seconds_delay': 3,
 'fpmm': '0x300D6D774188D301F1ce58C61FB630b8740b3C12',
 'maker_base_fee': 0,
 'taker_b